In [148]:
!pip install emoji

In [149]:
import pandas as pd
#we need to drop bad lines
namira_df = pd.read_csv("Artifical_inteligence_data.csv",on_bad_lines='skip')

In [150]:
namira_df.head()

,sentiment,text,user
0,positive,RT @cutedejun: sm not letting xiaojun go for h...,89SHAWDUHJ
1,neutral,@JDaniel2033 He spoke at my Naval boot camp gr...,muhlmannhoss
2,neutral,RT @MdBreathe: Here’s the man to thank for get...,SamArden3
3,negative,RT @mainintrovert: Aww nagtampo puro raw ako #...,Danielle_240702
4,negative,RT @devinisasleep: Lexi bout to be giving that...,jxdanquah


In [151]:
namira_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  99 non-null     object
 1   text       99 non-null     object
 2   user       99 non-null     object
dtypes: object(3)
memory usage: 2.4+ KB


In [152]:
namira_df.isnull().sum()

sentiment    0
text         0
user         0
dtype: int64

In [153]:
namira_df.shape

(99, 3)

In [154]:
namira_df['sentiment'].value_counts()

neutral     60
positive    26
negative    13
Name: sentiment, dtype: int64

Drop the user column.

In [155]:
namira_df.drop(['user'], axis=1, inplace = True)
namira_df

,sentiment,text
0,positive,RT @cutedejun: sm not letting xiaojun go for h...
1,neutral,@JDaniel2033 He spoke at my Naval boot camp gr...
2,neutral,RT @MdBreathe: Here’s the man to thank for get...
3,negative,RT @mainintrovert: Aww nagtampo puro raw ako #...
4,negative,RT @devinisasleep: Lexi bout to be giving that...
...,...,...
94,neutral,RT @MdBreathe: Here’s the man to thank for get...
95,neutral,RT @48RequestHour: AKB48 Team TP's Captain Che...
96,positive,RT @kookiejinnist: 23 yr old Uni student jk ce...
97,neutral,RT @MdBreathe: Here’s the man to thank for get...


In [156]:
namira_df['text']=namira_df['text']

In [157]:
namira_df['text'] = namira_df['text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),:]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [158]:
namira_df['text'] = namira_df['text'].str.lower()

Use regular expressions or python string methods to get rid of the additional data at the begging and end of each tweet

In [159]:
import string
punctuation_list =set({})
for i in range(0,len(namira_df.index)):
    divide = namira_df['text'][i].split(':')
    if len(divide) == 2:
      #remove text before :
      namira_df['text'][i] =  namira_df['text'][i].replace(divide[0],'')
    else:
      punctuation_list = punctuation_list | set(namira_df['text'][i]).intersection(set(string.punctuation))

In [160]:
#remove all punctuation
import string
for i in range(len(namira_df.index)):
  namira_df['text'][i] = namira_df['text'][i].translate(str.maketrans('', '', string.punctuation))
  #removing whitespaces
  namira_df['text'][i] = ' '.join([token for token in namira_df['text'][i].split()])

remove emoji

In [162]:
import emoji
def extract_emojis(s):
  return ''.join(c for c in s if c in emoji.UNICODE_EMOJI['en'])
for i in range(0,len(namira_df.index)):
    extract_emojis(namira_df['text'][i])

In [172]:
namira_df['text']

0     sm not letting xiaojun go for his graduation n...
1     jdaniel2033 he spoke at my naval boot camp gra...
2     here’s the man to thank for getting this march...
3     aww nagtampo puro raw ako kdlex sorry rui 🥺 st...
4     lexi bout to be giving that graduation speech ...
                            ...                        
94    here’s the man to thank for getting this march...
95    akb48 team tps captain chen shihya announced g...
96    23 yr old uni student jk celebrating graduatio...
97    here’s the man to thank for getting this march...
98    here’s the man to thank for getting this march...
Name: text, Length: 99, dtype: object

Add a column to your dataframe to reflect the length of each tweet, name it “tweet_len”.

In [163]:
namira_df['tweet_len'] = namira_df['text'].str.len()
namira_df

,sentiment,text,tweet_len
0,positive,sm not letting xiaojun go for his graduation n...,122
1,neutral,jdaniel2033 he spoke at my naval boot camp gra...,84
2,neutral,here’s the man to thank for getting this march...,120
3,negative,aww nagtampo puro raw ako kdlex sorry rui 🥺 st...,114
4,negative,lexi bout to be giving that graduation speech ...,120
...,...,...,...
94,neutral,here’s the man to thank for getting this march...,120
95,neutral,akb48 team tps captain chen shihya announced g...,101
96,positive,23 yr old uni student jk celebrating graduatio...,119
97,neutral,here’s the man to thank for getting this march...,120


Load the positive and negative words lexicons into two dataframe

In [164]:
pos1 = pd.read_csv("positive_words.txt",sep="\t",encoding='latin1',header=None)
neg1 = pd.read_csv("negative_words.txt",sep="\t",encoding='latin1',header=None)

pos1.columns = ["words"]
neg1.columns = ["words"]

pos_set = set(list(pos1["words"]))
neg_set = set(list(neg1["words"]))
print (len(pos_set))
print (len(neg_set))

2006
4783


Iterate through all of the words in each tweet and hit against the list of lexicons in the positive and negative word datframes. Since there are longer tweets, you need to normalize the number of positive and negative hits by the number of words in each tweet.

In [165]:
#Exclude empty rows

In [166]:
t1=namira_df
t2 = t1[t1.tweet_len>=1]
len(t1),len(t2)

(99, 99)

In [167]:
#take a sample of 10%
t3 = t2.sample(frac=0.1)
len(t2),len(t3)

(99, 10)

In [168]:
final_tag_list = []
pos_percent_list = []
neg_percent_list = []
pos_set_list = []
neg_set_list = []

for i,row in t3.iterrows():
    
    full_txt_set = set(row["text"].split())
    sent_len = len(full_txt_set)
    
    pos_set1 = (full_txt_set) & (pos_set)
    neg_set1 = (full_txt_set) & (neg_set)
    
    com_pos = len(pos_set1)
    com_neg = len(neg_set1)
    
    if(com_pos>0):
        pos_percent = com_pos/sent_len
    else:
        pos_percent = 0

    
    if(com_neg>0):
        neg_percent = com_neg/sent_len
    else:
        neg_percent =0
        
    if(pos_percent>0)|(neg_percent>0):
        if(pos_percent>neg_percent):
            final_tag = "positive"
        else:
            final_tag = "negative"
    else:
        final_tag="neutral"
    
    final_tag_list.append(final_tag)
    pos_percent_list.append(pos_percent)
    neg_percent_list.append(neg_percent)
    pos_set_list.append(pos_set1)
    neg_set_list.append(neg_set1)


In [169]:
t3.head(2)

,sentiment,text,tweet_len
52,neutral,vinodmeena966 ashwinivaishnaw railminindia इसम...,112
63,positive,jen spent time on facebook she enjoyed learnin...,116


Add two columns to your datafrme one to reflect the percentage of positiveness and the second to reflect the percentage of negativeness.

Tag each tweet with a sentiment score

In [170]:
t3["pred_sentiment_score"] = final_tag_list
t3["pos_percent"] = pos_percent_list
t3["neg_percent"] = neg_percent_list
t3["pos_set"] = pos_set_list
t3["neg_set"] = neg_set_list
t3.head()


,sentiment,text,tweet_len,pred_sentiment_score,pos_percent,neg_percent,pos_set,neg_set
52,neutral,vinodmeena966 ashwinivaishnaw railminindia इसम...,112,neutral,0.000000,0.0,{},{}
63,positive,jen spent time on facebook she enjoyed learnin...,116,positive,0.055556,0.0,{enjoyed},{}
13,positive,349 happy graduation,20,positive,0.333333,0.0,{happy},{}
93,negative,i wanted to be a pediatrician little me said b...,85,neutral,0.000000,0.0,{},{}
24,neutral,preview of some of kato rena’s upcoming gradua...,120,neutral,0.000000,0.0,{},{}


In [173]:
t3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 52 to 69
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   sentiment             10 non-null     object 
 1   text                  10 non-null     object 
 2   tweet_len             10 non-null     int64  
 3   pred_sentiment_score  10 non-null     object 
 4   pos_percent           10 non-null     float64
 5   neg_percent           10 non-null     float64
 6   pos_set               10 non-null     object 
 7   neg_set               10 non-null     object 
dtypes: float64(2), int64(1), object(5)
memory usage: 720.0+ bytes


Calculate the Accuracy and F1 score

In [174]:
from sklearn.metrics import accuracy_score
print (accuracy_score(t3["sentiment"],t3["pred_sentiment_score"]))

0.7


In [175]:
from sklearn.metrics import f1_score
f1_score(t3["sentiment"],t3["pred_sentiment_score"], average='macro')  

0.6534391534391535

In [176]:
rows_name = t3["sentiment"].unique()

from sklearn.metrics import confusion_matrix
cmat = pd.DataFrame(confusion_matrix(t3["sentiment"],t3["pred_sentiment_score"], labels=rows_name, sample_weight=None))
cmat.columns = rows_name 
cmat["act"] = rows_name
cmat

,neutral,positive,negative,act
0,2,0,0,neutral
1,1,4,0,positive
2,2,0,1,negative


In [178]:
### Accuracy without neutral 
t4 = t3.loc[(t3.sentiment!="neutral") & (t3.pred_sentiment_score!="neutral")].reset_index()
print (accuracy_score(t4["sentiment"],t4["pred_sentiment_score"]))
print (f1_score(t4["sentiment"],t4["pred_sentiment_score"],average='macro'))

1.0
1.0


In [179]:
cmat = pd.DataFrame(confusion_matrix(t4["sentiment"],t4["pred_sentiment_score"], labels=rows_name, sample_weight=None))
cmat.columns = rows_name 
cmat["act"] = rows_name
cmat

,neutral,positive,negative,act
0,0,0,0,neutral
1,0,4,0,positive
2,0,0,1,negative
